In [1]:
!pip install requests

In [1]:
# Importando as bibliotecas necessárias
import requests
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

In [110]:
# Fazendo a requisição da tabela de descrição das instituições financeiras usando a API do Banco Central
fin_institution = pd.DataFrame()
quarter = ['202209']
for i in quarter:
    link_insittutions = f"https://olinda.bcb.gov.br/olinda/servico/IFDATA/versao/v1/odata/IfDataCadastro(AnoMes=@AnoMes)?@AnoMes={i}&$top=10000&$format=json"
    request_fin_inst = requests.get(link_insittutions)
    info_json = request_fin_inst.json()
    fin_institution = pd.DataFrame(info_json['value']).append(fin_institution)
    print(i, 'fim')

202209 fim


C:\Users\xavie\AppData\Local\Temp\ipykernel_21492\3202647495.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  fin_institution = pd.DataFrame(info_json['value']).append(fin_institution)


In [111]:
# Selecionando as colunas com as funções da Pandas para criar nosso DataFrame desejado
# Agrupamos por Id e por Instuição para ter certeza que não haverá duplicatas
# Renomeamos algumas colunas
fin_institution = fin_institution[['CodInst', 'NomeInstituicao', 'Atividade', 'Municipio', 'Situacao']]
fin_institution = fin_institution.drop_duplicates(subset=['CodInst', 'NomeInstituicao'])
fin_institution = fin_institution.rename(columns={'CodInst': 'ID', 'NomeInstituicao': 'Inst Financeira' })
fin_institution

,ID,Inst Financeira,Atividade,Municipio,Situacao
0,C0041856,ABC-BRASIL,Crédito Atacado,São Paulo,A
1,C0080312,ABC-BRASIL - PRUDENCIAL,Crédito Atacado,São Paulo,A
2,C0052302,ABN AMRO,None,São Paulo,A
3,C0051956,AGIBANK,Crédito Varejo,Campinas,A
4,C0083694,AGIBANK - PRUDENCIAL,Crédito Varejo,Campinas,A
...,...,...,...,...,...
5186,00991143,WORLDPAY DO BRASIL INSTITUICAO DE PAGAMENTO LTDA,None,São Paulo,A
5187,C0086110,WORLDPAY DO BRASIL IP - PRUDENCIAL,None,São Paulo,A
5188,14338304,YAPAY PAGAMENTOS ONLINE LTDA,None,Marília,I
5189,19468242,ZOOP TECNOLOGIA & INSTITUICAO DE PAGAMENTO S.A.,None,Rio de Janeiro,I


In [112]:
# Fazendo a requisição dos dados resumidos utilizando a API do Banco Central
resumo = pd.DataFrame()
for i in quarter:
    resumo_link = f"https://olinda.bcb.gov.br/olinda/servico/IFDATA/versao/v1/odata/IfDataValores(AnoMes=@AnoMes,TipoInstituicao=@TipoInstituicao,Relatorio=@Relatorio)?@AnoMes={i}&@TipoInstituicao=2&@Relatorio='1'&$top=99999&$format=json"
    resumo_request = requests.get(resumo_link)
    resumo_json = resumo_request.json()
    resumo = pd.DataFrame(resumo_json['value']).append(resumo)
    print(i,'fim')

202209 fim


C:\Users\xavie\AppData\Local\Temp\ipykernel_21492\1764877128.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  resumo = pd.DataFrame(resumo_json['value']).append(resumo)


In [113]:
resumo.head()

,TipoInstituicao,CodInst,AnoMes,NomeRelatorio,NumeroRelatorio,Grupo,Conta,NomeColuna,DescricaoColuna,Saldo
0,2,C0052072,202209,Resumo,1,None,78185,Captações,[41000007] + [42000006] + [43000005] + [46000002],2.615190e+10
1,2,C0052072,202209,Resumo,1,None,78184,Passivo Circulante e Exigível a Longo Prazo e ...,[40000008] + [50000005],3.457375e+10
2,2,C0052072,202209,Resumo,1,None,78183,Carteira de Crédito Classificada,[31000000],2.371504e+10
3,2,C0052072,202209,Resumo,1,None,78182,Ativo Total,[10000007] + [20000004],3.965848e+10
4,2,C0052065,202209,Resumo,1,None,78187,Lucro Líquido,[70000009]+[80000006]-[81956001],-3.095899e+07


In [78]:
# Usando a biblioteca Pandas para manipular nosso DataFrame e tratar os dados
# Utilizamos um left Join para trazer a Coluna Inst Financeira do nosso DataFrame fin_insitution
# Para tratar nossa coluna de data utilizamos a função to_datetime
resumo = resumo[['CodInst', 'AnoMes', 'NomeColuna', 'Saldo']]
resumo= resumo.rename(columns={'CodInst': 'ID', 'AnoMes': 'Data'})
resumo = resumo.merge(fin_institution, on='ID', how='left')
resumo = resumo[['ID', 'Inst Financeira','Data', 'Atividade', 'Municipio', 'Situacao','NomeColuna', 'Saldo']]
resumo['Data'] = pd.to_datetime(resumo['Data'], format="%Y%m")

In [79]:
resumo.head()

,ID,Inst Financeira,Data,Atividade,Municipio,Situacao,NomeColuna,Saldo
0,05658367,COOPERATIVA DE ECONOMIA E CRÉDITO MÚTUO DOS EM...,2022-09-01,Mútuo PF sem conta corrente,Salvador,A,Carteira de Crédito Classificada,957617.74
1,05658367,COOPERATIVA DE ECONOMIA E CRÉDITO MÚTUO DOS EM...,2022-09-01,Mútuo PF sem conta corrente,Salvador,A,Passivo Circulante e Exigível a Longo Prazo e ...,75157.26
2,05658367,COOPERATIVA DE ECONOMIA E CRÉDITO MÚTUO DOS EM...,2022-09-01,Mútuo PF sem conta corrente,Salvador,A,Captações,0.00
3,05658367,COOPERATIVA DE ECONOMIA E CRÉDITO MÚTUO DOS EM...,2022-09-01,Mútuo PF sem conta corrente,Salvador,A,Patrimônio Líquido,1887875.79
4,05658367,COOPERATIVA DE ECONOMIA E CRÉDITO MÚTUO DOS EM...,2022-09-01,Mútuo PF sem conta corrente,Salvador,A,Lucro Líquido,23897.31


In [85]:
# Criamos uma tabela dinâmica para ter as contas contábeis em colunmas separadas e agrupada por Instituição Financeira
df_pivot = pd.pivot_table(resumo, index=['ID', 'Inst Financeira', 'Data'], columns='NomeColuna', values='Saldo')


['Ativo Total',
 'Captações',
 'Carteira de Crédito Classificada',
 'Lucro Líquido',
 'Passivo Circulante e Exigível a Longo Prazo e Resultados de Exercícios Futuros',
 'Patrimônio Líquido']

In [89]:
df_pivot.to_csv('pivotable.csv')
df_banks = pd.read_csv('pivotable.csv')
fin_institution_2 = fin_institution[['ID','Atividade', 'Municipio', 'Situacao']]
df_banks = df_banks.merge(fin_institution_2, on='ID', how='left')

In [91]:
df_banks.head()

,ID,Inst Financeira,Data,Ativo Total,Captações,Carteira de Crédito Classificada,Lucro Líquido,Passivo Circulante e Exigível a Longo Prazo e Resultados de Exercícios Futuros,Patrimônio Líquido,Atividade,Municipio,Situacao
0,00068987,COOPERATIVA DE CRÉDITO SICOOB ARACREDI LTDA - ...,2022-09-01,4.410657e+08,2.504884e+08,2.893505e+08,3492250.45,3.812163e+08,5.984935e+07,Crédito rural,Araguari,A
1,00075847,COOPERATIVA DE CRÉDITO UNICRED CENTRO-SUL LTDA...,2022-09-01,1.873334e+09,1.583965e+09,1.064019e+09,8647828.46,1.662850e+09,2.104844e+08,Mútuo com relevância em PJ,Criciúma,A
2,00106180,CENTRAL DAS COOPERATIVAS DE CRÉDITO DOS ESTADO...,2022-09-01,1.213534e+09,0.000000e+00,1.805835e+07,638809.77,1.131116e+09,8.241847e+07,None,Cuiabá,A
3,00122327,"SANTINVEST S.A. - CREDITO, FINANCIAMENTO E INV...",2022-09-01,2.981466e+08,1.533919e+08,2.862878e+08,2297712.79,1.601516e+08,1.379949e+08,"Sociedades de Credito, Financiamento e Investi...",Florianópolis,A
4,00129753,COOPERATIVA DE ECONOMIA E CRÉDITO MÚTUO DOS EM...,2022-09-01,1.638501e+06,0.000000e+00,8.102345e+05,5195.02,2.215350e+05,1.416966e+06,Mútuo PF sem conta corrente,Urucânia,A


In [92]:
# 1. Retorno sobre o Patrimônio Líquido (ROE)
df_banks["ROE"] = df_banks["Lucro Líquido"] / df_banks["Patrimônio Líquido"]

# 2. Retorno sobre Ativos (ROA)
df_banks["ROA"] = df_banks["Lucro Líquido"] / df_banks["Ativo Total"]
# 5. Índice de Liquidez
df_banks["Índice de Liquidez"] = df_banks["Ativo Total"] / df_banks["Passivo Circulante e Exigível a Longo Prazo e Resultados de Exercícios Futuros"]
#Índice de Solvência
df_banks["Índice de Solvência"] = df_banks["Ativo Total"] / (df_banks["Passivo Circulante e Exigível a Longo Prazo e Resultados de Exercícios Futuros"] + df_banks["Patrimônio Líquido"])

In [93]:
df_banks.head()

,ID,Inst Financeira,Data,Ativo Total,Captações,Carteira de Crédito Classificada,Lucro Líquido,Passivo Circulante e Exigível a Longo Prazo e Resultados de Exercícios Futuros,Patrimônio Líquido,Atividade,Municipio,Situacao,ROE,ROA,Índice de Liquidez,Índice de Solvência
0,00068987,COOPERATIVA DE CRÉDITO SICOOB ARACREDI LTDA - ...,2022-09-01,4.410657e+08,2.504884e+08,2.893505e+08,3492250.45,3.812163e+08,5.984935e+07,Crédito rural,Araguari,A,0.058351,0.007918,1.156996,1.0
1,00075847,COOPERATIVA DE CRÉDITO UNICRED CENTRO-SUL LTDA...,2022-09-01,1.873334e+09,1.583965e+09,1.064019e+09,8647828.46,1.662850e+09,2.104844e+08,Mútuo com relevância em PJ,Criciúma,A,0.041085,0.004616,1.126581,1.0
2,00106180,CENTRAL DAS COOPERATIVAS DE CRÉDITO DOS ESTADO...,2022-09-01,1.213534e+09,0.000000e+00,1.805835e+07,638809.77,1.131116e+09,8.241847e+07,None,Cuiabá,A,0.007751,0.000526,1.072865,1.0
3,00122327,"SANTINVEST S.A. - CREDITO, FINANCIAMENTO E INV...",2022-09-01,2.981466e+08,1.533919e+08,2.862878e+08,2297712.79,1.601516e+08,1.379949e+08,"Sociedades de Credito, Financiamento e Investi...",Florianópolis,A,0.016651,0.007707,1.861652,1.0
4,00129753,COOPERATIVA DE ECONOMIA E CRÉDITO MÚTUO DOS EM...,2022-09-01,1.638501e+06,0.000000e+00,8.102345e+05,5195.02,2.215350e+05,1.416966e+06,Mútuo PF sem conta corrente,Urucânia,A,0.003666,0.003171,7.396126,1.0


In [105]:
df_banks.to_csv('banks.csv', index = False)
df_banks.to_excel('banks.xlsx', index= False)

In [96]:
# 